# Scraping script for obtaining data on data related jobs in Malaysia

In [8]:
import time
#from selenium import webdriver
from bs4 import BeautifulSoup
#from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import requests
import re
import os
from lxml import etree
import lxml.html, lxml.html.clean
from urllib.parse import urlparse
from urllib.parse import parse_qs
import pandas as pd
from itertools import product
import time


#driver = webdriver.Chrome(ChromeDriverManager().install())
verbose_flag = True
# htmlSource = driver.page_source

In [ ]:
data_engineer_site = r'https://www.jobstreet.com.my/en/job-search/data-engineer-jobs/'
main_site=r"https://www.jobstreet.com.my"

page = requests.get(data_engineer_site)

soup = BeautifulSoup(page.content, 'html.parser')



In [ ]:
#find number of result pages
results = soup.find('select', attrs={"id":"pagination"})
options = results.find_all('option')

num_result_pages = int(options[-1]['value'])

In [ ]:
jobslist = soup.find('div', attrs={'id':"jobList"}).find('div', attrs={'data-automation':'jobListing'})

In [ ]:
jobs = jobslist.find_all('a', attrs={'class':'_1hr6tkx5 _1hr6tkx8 _1hr6tkxb sx2jih0 sx2jihf zcydq8h', 'href': re.compile(r'^\/en\/job\/')})

In [ ]:
joblink = jobs[29].get('href')

In [ ]:
job_fulllink = main_site + joblink
#job_fulllink = r"https://www.jobstreet.com.my/en/job/data-engineer-4834504?jobId=jobstreet-my-job-4834504&sectionRank=1&token=0~9d2b2714-12eb-4e27-b40c-24f3babd0c52&fr=SRP%20View%20In%20New%20Tab"
job_page = requests.get(job_fulllink)

job_soup = BeautifulSoup(job_page.content, 'html.parser')


In [ ]:
parsed_url = urlparse(job_fulllink)
captured_value = parse_qs(parsed_url.query)
print(captured_value['jobId'])

In [ ]:
jobs_dom = etree.HTML(str(job_soup))

In [ ]:
job_title = jobs_dom.xpath('//*[@id="contentContainer"]/div/div[1]/div[1]/div[1]/div/div/div[1]/div/div/div[2]/div/div/div/div[1]/h1')[0].text
company_name = jobs_dom.xpath('//*[@id="contentContainer"]/div/div[1]/div[1]/div[1]/div/div/div[1]/div/div/div[2]/div/div/div/div[2]/span')[0].text
salary_range = jobs_dom.xpath('//*[@id="contentContainer"]/div/div[1]/div[1]/div[1]/div/div/div[2]/div/div/div/div[2]/span')[0].text

cleaner = lxml.html.clean.Cleaner(allow_tags=[''])
cleaned = cleaner.clean_html(etree.tostring(jobs_dom.xpath('//*[@id="contentContainer"]/div/div[2]/div/div[1]/div/div[1]/div/div[2]/div/span/div')[0]))
print()
job_description = cleaned.decode("utf-8").replace('<div>', '').replace('</div>','')
#job_description = etree.tostring(jobs_dom.xpath('//*[@id="contentContainer"]/div/div[2]/div/div[1]/div/div[2]/div/div[2]/div/span/div')[0])
# job_level = jobs_dom.xpath('//*[@id="contentContainer"]/div/div[2]/div/div[1]/div/div[2]/div/div[2]/div/div/div[1]/div/div/div[2]/span')[0].text
# years_experience = jobs_dom.xpath('//*[@id="contentContainer"]/div/div[2]/div/div[1]/div/div[2]/div/div[2]/div/div/div[3]/div/div/div[2]/span')[0].text
# qualifications = jobs_dom.xpath('//*[@id="contentContainer"]/div/div[2]/div/div[1]/div/div[2]/div/div[2]/div/div/div[3]/div/div/div[2]/span')[0].text
# job_type = jobs_dom.xpath('//*[@id="contentContainer"]/div/div[2]/div/div[1]/div/div[2]/div/div[2]/div/div/div[4]/div/div/div[2]/span')[0].text
# company_registration = jobs_dom.xpath('//*[@id="contentContainer"]/div/div[2]/div/div[2]/div[2]/div[2]/div/div[2]/div/div/div[1]/div/div/div[2]/span')[0].text
# company_size = jobs_dom.xpath('//*[@id="contentContainer"]/div/div[2]/div/div[2]/div[2]/div[2]/div/div[2]/div/div/div[1]/div/div/div[2]/span')[0].text
# industry_type = jobs_dom.xpath('//*[@id="contentContainer"]/div/div[2]/div/div[2]/div[2]/div[2]/div/div[2]/div/div/div[1]/div/div/div[2]/span')[0].text
# location = jobs_dom.xpath('//*[@id="contentContainer"]/div/div[1]/div[1]/div[1]/div/div/div[2]/div/div/div/div[1]/div/span')[0].text

if verbose_flag:
    print('job: ', job_title)
    print('company: ', company_name)
    print("salary range: ", salary_range)
    print('job description: ', job_description)



In [ ]:
for job in jobs_dom.xpath('//*[@id="contentContainer"]/div/div[2]/div/div[1]/div/div[1]/div/div[2]/div/span/div/div'):
    print(etree.tostring(job))

In [ ]:
cleaner = lxml.html.clean.Cleaner(allow_tags=[''])
cleaned = cleaner.clean_html(etree.tostring(jobs_dom.xpath('//*[@id="contentContainer"]/div/div[2]/div/div[1]/div/div[1]/div/div[2]/div/span/div/div')[0]))
print(cleaned.decode("utf-8").replace('<div>', '').replace('</div>',''))

In [ ]:
print(etree.tostring(job_description))

In [ ]:
job_title = employer_details.find('h1', attrs={'class':'sx2jih0 _18qlyvc0 _18qlyvch _1d0g9qk4 _18qlyvcp _18qlyvc1x'}).text
print(job_title)
company = employer_details.find('span', attrs={'class':'sx2jih0 zcydq84u _18qlyvc0 _18qlyvc1x _18qlyvc2 _1d0g9qk4 _18qlyvcb'}).text
print(company)
salary_range = employer_details.findAll('span', attrs={'class':'sx2jih0 zcydq84u _18qlyvc0 _18qlyvc1x _18qlyvc1 _18qlyvca'}).
print(salary_range)

# Definition of scraping functions

In [7]:
def search(keyword:str, region:str, retry = 0) -> list:
    '''
        search jobstreet for keyword, returns results as a list of job results
    '''
    try:
        if(retry > 20):
            raise Exception("Too many retries, endpoint notreachable")

        search_term = keyword.replace(' ', '-')
        root_link = r'https://www.jobstreet.com.my/en/job-search/'
        search_link = root_link + search_term + '-jobs' + "-in-" + region
        page = requests.get(search_link)
        if(page.status_code != 200):
                time.sleep(2)
                return search(keyword, region, retry + 1)
        soup = BeautifulSoup(page.content, 'html.parser')
        results = soup.find('select', attrs={"id":"pagination"})
        if(results is not None):
            options = results.find_all('option')
            num_result_pages = int(options[-1]['value'])
        else:
            num_result_pages = 1
        print(f"searching @ {search_link}")
        print(f"total results pages for search {keyword}:", {str(num_result_pages)})

        all_results = []
        print("scraping page 1...")
        first_page_results = get_job_links(search_link)
        all_results = all_results + first_page_results

        #get the rest of the result pages
        if(num_result_pages > 1):
            for page in range(2, num_result_pages+1):
                print(f"scraping page {page}...")
                search_page = search_link + '/' + str(page)
                results_job_links = get_job_links(search_page)
                all_results = all_results + results_job_links

        print("done!")
        return(all_results)

    except  Exception as e:
        print(e)
    


def get_job_links(page_link:str, retry=0) -> list:
    try:
        if(retry > 20):
            raise Exception("Too many retries, endpoint notreachable")

        results_job_links = []
        page = requests.get(page_link)
        if(page.status_code != 200):
                time.sleep(2)
                return get_job_links(page_link, retry + 1)
        soup = BeautifulSoup(page.content, 'html.parser')
        if(soup is None):
            return None
        jobs_list = soup.find('div', attrs={'id':"jobList"}).find('div', attrs={'data-automation':'jobListing'})
        jobs = jobs_list.find_all('a', attrs={'class':'_1hr6tkx5 _1hr6tkx8 _1hr6tkxb sx2jih0 sx2jihf zcydq8h', 'href': re.compile(r'^\/en\/job\/')})
        for job in jobs:
            job_link = job.get('href')
            results_job_links.append(job_link)

        return results_job_links

    except Exception as e:
        print(e)


def extract_job_details(job_page_link:str, retry = 0)->dict:
    try:
        if(retry > 20):
            raise Exception("Too many retries, endpoint notreachable")

        root_link = r"https://www.jobstreet.com.my"
        full_link = root_link + job_page_link
        parsed_url = urlparse(full_link)
        captured_value = parse_qs(parsed_url.query)
        
        jobID = captured_value['jobId']
        
        job_page = requests.get(full_link)
        if(job_page.status_code != 200):
            time.sleep(2)
            return extract_job_details(job_page_link, retry+1)
        job_soup = BeautifulSoup(job_page.content, 'html.parser')
        if(job_soup is None):
            raise Exception("No results")
        
        jobs_dom = etree.HTML(str(job_soup))
        job_title = jobs_dom.xpath('//*[@id="contentContainer"]/div/div[1]/div[1]/div[1]/div/div/div[1]/div/div/div[2]/div/div/div/div[1]/h1')[0].text
        salary_range = jobs_dom.xpath('//*[@id="contentContainer"]/div/div[1]/div[1]/div[1]/div/div/div[2]/div/div/div/div[2]/span')[0].text

        cleaner = lxml.html.clean.Cleaner(allow_tags=[''])
        cleaned = cleaner.clean_html(etree.tostring(jobs_dom.xpath('//*[@id="contentContainer"]/div/div[2]/div/div[1]/div')[0]))
                                                                    #//*[@id="contentContainer"]/div/div[2]/div/div[1]/div/div[1]/div/div[2]/div/span/div
        job_description = cleaned.decode("utf-8").replace('<div>', '').replace('</div>','')
        job_level = job_soup.find('span', string="Career Level")
        if(job_level is not None):
            job_level = list(list(job_level.parent.parent.children)[1].children)[0].text    

        experience = job_soup.find('span', string="Years of Experience")
        if (experience is not None):                 
            experience = list(list(experience.parent.parent.children)[1].children)[0].text    

        job_type = job_soup.find('span', string="Job Type")
        if(job_type is not None):
            job_type = list(list(job_type.parent.parent.children)[1].children)[0].text 

        qualifications = job_soup.find('span', string="Qualification")
        if(qualifications is not None):
            qualifications = list(list(qualifications.parent.parent.children)[1].children)[0].text

        job_specialization = job_soup.find('span', string="Job Specializations")
        if(job_specialization is not None):
            job_specialization = list(list(job_specialization.parent.parent.children)[1].children)[0].text


        company_name = jobs_dom.xpath('//*[@id="contentContainer"]/div/div[1]/div[1]/div[1]/div/div/div[1]/div/div/div[2]/div/div/div/div[2]/span')[0].text
        company_registration = job_soup.find('span', string="Registration No.")
        if(company_registration is not None):
            company_registration = list(list(company_registration.parent.parent.children)[1].children)[0].text

        company_size = job_soup.find('span', string="Company Size")
        if(company_size is not None):
            company_size = list(list(company_size.parent.parent.children)[1].children)[0].text

        company_industry = job_soup.find('span', string="Industry")
        if(company_industry is not None):
            company_industry = list(list(list(company_industry.parent.parent.children)[1].children)[0].children)[0].text
        
        return {"job_id":jobID, 'job_title': job_title, "salary_range":salary_range, "job_description":job_description, "job_level":job_level, "experience":experience, "job_type":job_type, "qualifications":qualifications, 'job_specialization':job_specialization, 'company_name':company_name, 'company_registration':company_registration, "company_size":company_size, "company_industry":company_industry}
    
    except Exception as e:
        print('exception caught!', str(e))
        print(job_page.status_code)
        return {"job_id":None, 'job_title': None, "salary_range":None, "job_description":None, "job_level":None, "experience":None, "job_type":None, "qualifications":None, 'job_specialization':None, 'company_name':None, 'company_registration':None, "company_size":None, "company_industry":None}

def get_job_page(job_page_link:str):
    root_link = r"https://www.jobstreet.com.my"
    full_link = root_link + job_page_link
    job_page = requests.get(full_link)
    job_soup = BeautifulSoup(job_page.content, 'html.parser')
    parsed_url = urlparse(job_fulllink)
    captured_value = parse_qs(parsed_url.query)
    
    job_ID = captured_value['jobId']

    return {'job_ID': job_ID, 'page':page.content}
    


In [ ]:
df = pd.DataFrame(columns=['a', 'b', 'c'])

data1 = {'a':1, 'b':2, 'c':3}
data2 = {'a':4, 'b':5, 'c':6}

df = df.append(data1, ignore_index=True)
df = df.append(data2, ignore_index=True)
df

In [9]:
#states = ["negeri-sembilan", "kedah", "pahang", "perak", "selangor", "sabah", "johor", "sarawak", "terengganu", "perlis", "kelantan", "melaka", "penang", "kuala-lumpur"]
states = ["selangor"]
#job_search_terms = ["data engineer", "data scientist", "data analyst"]
job_search_terms = ['data scientist']

columns = ["job_id", 'job_title', "salary_range", "job_description", "job_level", "experience", "job_type", "qualifications", 'job_specialization', 'company_name', 'company_registration', "company_size", "company_industry", "job_link", "search_term", "state"]
jobs_df = pd.DataFrame(columns=columns)


for search_term, state in product(job_search_terms, states):
    print(search_term, state)
    results = search(search_term, state)

    for job_link in results:
        time.sleep(0.2)
        job_details = extract_job_details(job_link)
        job_details['job_link'] = job_link
        job_details['search_term'] = search_term
        job_details['state'] = state

        jobs_df = jobs_df.append(job_details, ignore_index=True)


data scientist selangor
searching @ https://www.jobstreet.com.my/en/job-search/data-scientist-jobs-in-selangor
total results pages for search data scientist: {'25'}
scraping page 1...
scraping page 2...
scraping page 3...
scraping page 4...
scraping page 5...
scraping page 6...
scraping page 7...
scraping page 8...
scraping page 9...
scraping page 10...
scraping page 11...
scraping page 12...
scraping page 13...
scraping page 14...
scraping page 15...
scraping page 16...
scraping page 17...
scraping page 18...
scraping page 19...
scraping page 20...
scraping page 21...
scraping page 22...
scraping page 23...
scraping page 24...
scraping page 25...
done!


C:\Users\RSYeo\AppData\Local\Temp\ipykernel_6124\1759806109.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  jobs_df = jobs_df.append(job_details, ignore_index=True)
C:\Users\RSYeo\AppData\Local\Temp\ipykernel_6124\1759806109.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  jobs_df = jobs_df.append(job_details, ignore_index=True)
C:\Users\RSYeo\AppData\Local\Temp\ipykernel_6124\1759806109.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  jobs_df = jobs_df.append(job_details, ignore_index=True)
C:\Users\RSYeo\AppData\Local\Temp\ipykernel_6124\1759806109.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  jobs_df = jobs_df.app

In [10]:
path = r"C:\Users\RSYeo\Projects\data\data_scientist2_2022_03_02.ftr"
jobs_df.to_feather(path)

In [ ]:
jobs_df2 = pd.read_feather(path)

In [ ]:
jobs_df.shape


In [ ]:
JD = extract_job_details(results[1])
JD2 = extract_job_details(results[3])


In [ ]:
df = pd.DataFrame(JD)

In [ ]:
df = df.append(JD2, ignore_index=True)

In [ ]:
df.head(50)

In [ ]:
page = requests.get(r"https://www.jobstreet.com.my/en/job/assistant-vice-president-data-engineer-compliance-amlt-remediation-4824511?fr=SRP%20Job%20Listing&jobId=jobstreet-my-job-4824511&sectionRank=2&token=0~2cdf76e9-a859-4c15-bb9a-e16a948b507a")

In [ ]:
soup = BeautifulSoup(page.content, 'html.parser')
list(list(soup.find('span', string="Career Level").parent.parent.children)[1].children)[0].text

In [ ]:
extract_job_details('/en/job/data-scientist-9316926/origin/sg?jobId=jobstreet-sg-job-9316926&sectionRank=30&token=0~474bcf85-700c-437e-8294-235da1e78670&fr=SRP%20Job%20Listing')

# Cleaning Data

### Load data


In [2]:
path_data_scientist = r"C:\Users\RSYeo\Projects\data\data_scientist_2022_03_02.ftr"
path_data_engineer =r"C:\Users\RSYeo\Projects\data\data_engineer_2022_03_02.ftr"
path_data_analyst = r"C:\Users\RSYeo\Projects\data\data_analyst_2022_03_02.ftr"

In [5]:
df_data_scientist = pd.read_feather(path_data_scientist)
df_data_scientist.head()

,job_id,job_title,salary_range,job_description,job_level,experience,job_type,qualifications,job_specialization,company_name,company_registration,company_size,company_industry,job_link,search_term,state
0,[jobstreet-my-job-4839088],Big Data Lead (Work in Singapore) - 44565,"SGD 7,000 - SGD 10,500",Job HighlightsExciting future with breakthroug...,Manager,7 years,Full-Time,"Bachelor's Degree, Post Graduate Diploma, Prof...","Computer/Information Technology, IT-Software",Keysight Technologies Malaysia Sdn. Bhd.,463532-M,2001 - 5000 Employees,Electrical & Electronics,/en/job/big-data-lead-work-in-singapore-44565-...,data scientist,negeri-sembilan
1,[jobstreet-my-job-4832121],SQL Database Administrator (数据库管理员),"MYR 15,000 - MYR 25,000","Job DescriptionJob Requirements: Must speak, r...",Senior Executive,3 years,Full-Time,"Bachelor's Degree, Post Graduate Diploma, Prof...","Computer/Information Technology, IT-Network/Sy...",Morivy Data and Technology Inc.,None,1 - 50 Employees,Computer / Information Technology (Software),/en/job/sql-database-administrator-%E6%95%B0%E...,data scientist,negeri-sembilan
2,[jobstreet-my-job-4831309],Korean Content Specialist (Pharmaceutical) Bas...,Posted on 24-Feb-22,Job HighlightsBest Workplace EverCompetitive B...,Senior Executive,1 year,Full-Time,"Bachelor's Degree, Post Graduate Diploma, Prof...","Others, Publishing",Clarivate (Malaysia) Sdn Bhd,1360469-D,2001 - 5000 Employees,"Consulting (IT, Science, Engineering & Technical)",/en/job/korean-content-specialist-pharmaceutic...,data scientist,negeri-sembilan
3,[jobstreet-my-job-4852008],Manufacturing Engineer- Fresh Graduates (1 yea...,"MYR 3,000 - MYR 3,500",Job HighlightsCareer AdvancementDaily Meal All...,Entry Level,None,Full-Time,"Bachelor's Degree, Post Graduate Diploma, Prof...","Engineering, Electrical, Mechanical",Plexus Manufacturing Sdn Bhd,399136-M,More than 5000 Employees,Electrical & Electronics,/en/job/manufacturing-engineer-fresh-graduates...,data scientist,negeri-sembilan
4,[jobstreet-my-job-4852655],Production Executive,Posted 12 hours ago,Job DescriptionJob Summary Responsible for the...,Senior Executive,3 years,Full-Time,"Diploma, Advanced/Higher/Graduate Diploma, Bac...","Manufacturing, Manufacturing",SUNSHINE BREAD SDN BHD (Formerly known as Auri...,1219557K,51 - 200 Employees,Manufacturing / Production,/en/job/production-executive-4852655?jobId=job...,data scientist,negeri-sembilan


### Start cleaning data

In [6]:
# Format job id to be numerical
df_data_scientist[]